##Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [4]:
import os
BASE_DIR = '/content/drive/MyDrive/Colab Notebooks/Deep-Learning-LW3'
DATA_DIR = os.path.join(BASE_DIR, "scraped_wiki_data")

##Wikipedia Api usage

In [9]:
!pip3 install wikipedia-api
import wikipediaapi
import random
import time
import sys
from os.path import exists

In [25]:
class Wikipedia_scraper:
  scraped_count = 0
  retry_count = 0
  pages_for_backup = []

  def __init__(self, wiki_lang, timeout_in_secs_for_exception, min_links_for_valid_page, min_char_count_for_valid_page, pages_to_scrape_count):
    self.wiki_api = wikipediaapi.Wikipedia(wiki_lang)
    self.timeout_for_exception = timeout_in_secs_for_exception
    self.min_links_for_valid_page = min_links_for_valid_page
    self.min_char_count_for_valid_page = min_char_count_for_valid_page
    self.left_to_scrape_count = pages_to_scrape_count

  # method that scrapes a page, takes next random link from the page and scrapes the next page
  def ScrapeRandomNextPages(self, start_page_title):
    curr_page = self.wiki_api.page(start_page_title)
    last_title = start_page_title

    try:
      while( self.left_to_scrape_count > 0 ):
        print(str(self.scraped_count) + ". " + curr_page.title)
        curr_page_text = curr_page.text
        curr_page_title = curr_page.title
        last_title = curr_page.title 
        if (curr_page_text != None and curr_page_title != None):
          file_name = curr_page_title.replace("/", "_")
          file_name = file_name.replace("\\", "_")
          new_file_dir = os.path.join(DATA_DIR, file_name + ".txt")
          if (not exists(new_file_dir)):
            with open(new_file_dir, 'w') as writefile:
              writefile.write(curr_page_text)
            self.scraped_count += 1
            self.left_to_scrape_count += 1
        curr_page = self.GetNextValidPage(curr_page)
        self.AddBackupPages(curr_page)
        
        self.retry_count = 0
    except FileNotFoundError as e:
      raise e
    except:
      self.retry_count += 1
      if (self.retry_count > 10):
        raise sys.exc_info()
      time.sleep(self.timeout_for_exception)
      self.ScrapeRandomNextPages(last_title)

  def AddBackupPages(self, page):
    while( len(self.pages_for_backup) < 10 ):
      self.pages_for_backup.append(page)

  def GetNextValidPage(self, curr_page):
    curr_page_links = list(curr_page.links)
    next_page = None

    random.shuffle(curr_page_links)

    for link in curr_page_links:
      page = self.GetPageFromLink(link)
      if ( self.IsPageValid(page) and page != None):
        next_page = page
        break

    if( next_page == None):
      print("Backing up to page from backup")
      return self.pages_for_backup.pop()

    return next_page

  def IsPageValid(self, page):
    if( page == None ):
      return False
    if( not page.exists() ):
      return False
    if(page.links == None):
      return False
    if( len(page.links) < self.min_links_for_valid_page):
      return False
    if ( len(page.text) < self.min_char_count_for_valid_page ):
      return False
    return True

  def GetPageFromLink(self, link):
    link_no_spaces = link.replace(" ", "_")
    page = self.wiki_api.page(link_no_spaces)
    return page

scraper = Wikipedia_scraper('lt', 60, 10, 500, 6000)
scraper.ScrapeRandomNextPages('Lietuvos_upės')

Streaming output truncated to the last 5000 lines.
3773. Tekstilė
3773. Elektrotechnika
3774. Amunicija
3775. Ginklas
3776. Branduolinis ginklas
3776. Arsenalas
3777. Kanaletas
3778. Giovanni Paolo Panini
3779. Bernardo Bellotto
3780. 1744 m.
3780. Mykolas Pranciškus Karpavičius
3781. 1744 m.
3781. Prancūzija
3781. 1958 m. Prancūzijos Konstitucija
3782. 1992 m.
3782. 1962 m.
3782. Gintaras Ramonas
3782. Bariūnai
3783. 1950 m.
3783. Vasario 18
3783. Vilniaus universitetas
3783. 1976 m.
3783. Lapkričio 23
3783. Penktadienis
3783. Sekmadienis
3783. Gegužė
3783. Spaudos atgavimo, kalbos ir knygos diena
3784. Vikipedija:Šaltinių nurodymas
3784. Vikipedijos aptarimas:Šaltinių nurodymas
3784. Kerpena
3784. Karalystė (biologija)
3784. Tipas
3784. Karalystė (biologija)
3784. Taksonas
3784. Drugiai
3784. Pūkanugariai
3785. Alksnis
3786. Medis
3786. Sąrašas:Lietuvos svetimžemė dendroflora
3787. Kanada
3787. Centrinė Amerika
3788. Vikipedija:Patikrinamumas
3788. Vikipedija:Puslapių apsaugos taisyk

KeyboardInterrupt: ignored

##GMM

In [ ]:
import cv2
import albumentations as A
import glob
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
import matplotlib.patches as patches
import copy
import time
import random
import csv
from skimage import io, transform
import numpy as np
import pickle
import io
import matplotlib.pyplot as plt
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor